In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import nltk
from sklearn import *

test = pd.read_csv('../input/test_stage_1.tsv', delimiter='\t').rename(columns={'A': 'A_Noun', 'B': 'B_Noun'})
sub = pd.read_csv('../input/sample_submission_stage_1.csv')
test.shape, sub.shape

((2000, 9), (2000, 4))

In [2]:
test['mask'] = test['Text'].map(lambda x: '*' * len(x))
test['mask'] = test.apply(lambda r: r['mask'][: r['Pronoun-offset']] + 'P' * len(str(r['Pronoun'])) + r['mask'][r['Pronoun-offset'] + len(str(r['Pronoun'])): ], axis=1)
test['mask'] = test.apply(lambda r: r['mask'][: r['A-offset']] + 'A' * len(str(r['A_Noun'])) + r['mask'][r['A-offset'] + len(str(r['A_Noun'])): ], axis=1)
test['mask'] = test.apply(lambda r: r['mask'][: r['B-offset']] + 'B' * len(str(r['B_Noun'])) + r['mask'][r['B-offset'] + len(str(r['B_Noun'])): ], axis=1)
test['section_min'] = test[['Pronoun-offset', 'A-offset', 'B-offset']].min(axis=1)
test['Pronoun-offset2'] = test['Pronoun-offset'] + test['Pronoun'].map(len)
test['A-offset2'] = test['A-offset'] + test['A_Noun'].map(len)
test['B-offset2'] = test['B-offset'] + test['B_Noun'].map(len)                               
test['section_max'] = test[['Pronoun-offset2', 'A-offset2', 'B-offset2']].max(axis=1)
print(test['Text'][0][test['section_min'][0]:test['section_max'][0]])
print(test['mask'][0][test['section_min'][0]:test['section_max'][0]])

Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her
AAAAAAAAAAAAAA**BBBBBBB************************************************************PPP


In [4]:
doc = nlp(test['Text'][0][test['section_min'][0]:test['section_max'][0]])
tokens = pd.DataFrame(
    [[token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, [child for child in token.children]] for token in doc],
    columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'is_alpha', 'is_stop', 'child'])
tokens

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop,child
0,Cheryl,cheryl,PROPN,NNP,compound,Xxxxx,True,False,[]
1,Cassidy,cassidy,PROPN,NNP,ROOT,Xxxxx,True,False,"[Cheryl, ,, friend, and, pupil, .]"
2,",",",",PUNCT,",",punct,",",False,False,[]
3,Pauline,pauline,PROPN,NNP,poss,Xxxxx,True,False,['s]
4,'s,'s,PART,POS,case,'x,False,False,[]
5,friend,friend,NOUN,NN,appos,xxxx,True,False,[Pauline]
6,and,and,CCONJ,CC,cc,xxx,True,True,[]
7,also,also,ADV,RB,advmod,xxxx,True,True,[]
8,a,a,DET,DT,det,x,True,True,[]
9,year,year,NOUN,NN,nmod,xxxx,True,False,[11]


In [ ]:
test['A'] = test.apply(lambda r: 0.45 if r['A-offset'] == r['section_min'] else 0.4, axis=1)
test['B'] = test.apply(lambda r: 0.3 if r['B-offset'] == r['section_min'] else 0.45, axis=1)
test['NEITHER'] = 1.0 - (test['A'] + test['B'])

In [8]:
test

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,B_Noun,B-offset,URL,mask,section_min,Pronoun-offset2,A-offset2,B-offset2,section_max
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...,**********************************************...,191,277,205,214,277
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie,**********************************************...,228,287,237,264,287
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...,**********************************************...,173,268,180,256,268
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band),**********************************************...,174,324,178,351,351
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera,**********************************************...,219,440,236,300,440
5,development-6,Sandra Collins is an American DJ. She got her ...,She,411,Collins,236,DJ,347,http://en.wikipedia.org/wiki/Sandra_Collins,**********************************************...,236,414,243,349,414
6,development-7,Reb Chaim Yaakov's wife is the sister of Rabbi...,his,273,Reb Asher,152,Akiva Eiger,253,http://en.wikipedia.org/wiki/Asher_Arieli,**********************************************...,152,276,161,264,276
7,development-8,Slant Magazine's Sal Cinquemani viewed the alb...,his,337,Greg Kot,173,Robert Christgau,377,http://en.wikipedia.org/wiki/The_Truth_About_L...,**********************************************...,173,340,181,393,393
8,development-9,Her father was an Englishman ``of rank and cul...,her,246,Mary Paine,255,Kelsey,267,http://en.wikipedia.org/wiki/Mary_S._Peake,**********************************************...,246,249,265,273,273
9,development-10,Shaftesbury's UK partners in the production of...,she,329,Christina Jennings,196,Kirstine Stewart,226,http://en.wikipedia.org/wiki/Murdoch_Mysteries,**********************************************...,196,332,214,242,332
